In [44]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

In [ ]:

import os

load_dotenv()
your_key = os.getenv("YOUR_KEY_NAME")


In [46]:
openai_api_key = os.getenv("OPENAI_API_KEY")
deployment = os.getenv("DEPLOYMENT_NAME")
endpoint = os.getenv("ENDPOINT_URL")

In [51]:

client = AzureOpenAI(
    api_key=openai_api_key,  
    api_version="2024-05-01-preview",
    azure_endpoint = endpoint
    )
    
deployment_name=deployment #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment. 
    
# Send a completion call to generate an answer
print('Sending a test completion job')
start_phrase = 'Write a tagline for a Church.'
response = client.completions.create(model=deployment_name, prompt=start_phrase, max_tokens=50)
print(start_phrase+response.choices[0].text)

Sending a test completion job
Write a tagline for a Church. We have a dozen churches in my community. Understanding a little bit about what the church stands for will give you a TON of information to work from. I’m going to make something up off the top of my head to give you an example.


In [52]:
response = client.chat.completions.create(
    model = deployment_name,
    messages=[{"role":"user", "content": "is it too late to join the course?"}]
)

In [53]:
response.choices[0].message.content

'As an AI language model, I do not have access to information about specific courses or their enrollment deadlines. Please contact the course provider or educational institution directly for information regarding enrollment deadlines or possible options for joining the course.'

### LLM Zoomcamp 1.3 -  Retrieval and Search

In [5]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-08-15 21:45:35--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-08-15 21:45:35 (39.0 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [8]:
!wget https://raw.githubusercontent.com/DrUkachi/llm-zoomcamp/main/01-intro/documents.json

--2024-08-15 21:49:07--  https://raw.githubusercontent.com/DrUkachi/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-08-15 21:49:07 (65.2 MB/s) - ‘documents.json’ saved [658332/658332]



In [9]:
import minsearch
import json

In [ ]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

In [23]:
docs_raw[1]['course']

'machine-learning-zoomcamp'

In [11]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

In [26]:
q = "the course has already started, can I still enroll?"

In [27]:
index.fit(documents)

In [30]:
boost = {'question': 3.0}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5

)

In [31]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

### LLM Zoomcamp 1.4 - Generating Answers with OpenAI GPT 4.0

In [54]:
response = client.chat.completions.create(
    model = deployment_name,
    messages=[{"role":"user", "content": q}]
)

In [55]:
response.choices[0].message.content

'As an AI language model, I am unable to provide information about a specific course or institution. However, typically the decision to allow enrollment after a course has started is up to the discretion of the institution or instructor. It is recommended that you reach out to the institution directly to inquire about their policies regarding enrollment after a course has already begun.'

In [57]:
prompt_template = """

You are a course teaching assistant. Answer the Question based on the context. 
Use only the facts from the Context when answering the questions.
If the context does not contain the answer say "You don't have answer to the question."
Question: {question}
Context: {context}

"""

In [61]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

context

"section: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection: General course-related questions\nquestion: Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.\n\nsection: General course-related questions\nquestion: Course - When will the course start?\nanswer: The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course 